# EEG Signal Analysis - KP Code

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq
from scipy import stats

## Key Computational Techniques

- Numpy for numerical computations
- Scipy for signal processing
- Matplotlib for visualization
- Pandas for data manipulation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
combined_df = pd.read_csv("/content/drive/MyDrive/Katie Programming Folder/20241127_brainwaves.csv")

Mounted at /content/drive


In [ ]:
print(combined_df.columns)

Index(['TimeStamp', 'Delta_TP9', 'Delta_AF7', 'Delta_AF8', 'Delta_TP10',
       'Theta_TP9', 'Theta_AF7', 'Theta_AF8', 'Theta_TP10', 'Alpha_TP9',
       'Alpha_AF7', 'Alpha_AF8', 'Alpha_TP10', 'Beta_TP9', 'Beta_AF7',
       'Beta_AF8', 'Beta_TP10', 'Gamma_TP9', 'Gamma_AF7', 'Gamma_AF8',
       'Gamma_TP10', 'RAW_TP9', 'RAW_AF7', 'RAW_AF8', 'RAW_TP10', 'AUX_RIGHT',
       'Mellow', 'Concentration', 'Accelerometer_X', 'Accelerometer_Y',
       'Accelerometer_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z', 'HeadBandOn',
       'HSI_TP9', 'HSI_AF7', 'HSI_AF8', 'HSI_TP10', 'Battery', 'Delta_PCA',
       'Theta_PCA', 'Alpha_PCA', 'Beta_PCA', 'Gamma_PCA', 'Delta_Median',
       'Theta_Median', 'Alpha_Median', 'Beta_Median', 'Gamma_Median'],
      dtype='object')


# Fourier Transformation

In [ ]:
def advanced_signal_analysis(combined_df, sample_percentage=0.25, random_state=55):
    """
    Comprehensive EEG signal analysis with Fourier transformation and advanced visualizations

    Parameters:
    -----------
    combined_df : pandas DataFrame
        Original EEG dataframe
    sample_percentage : float, optional
        Percentage of data to sample
    random_state : int, optional
        Random seed for reproducibility

    Returns:
    --------
    analysis_results : dict
        Comprehensive analysis results
    """
    # Sampling
    sample_size = int(len(combined_df) * sample_percentage)
    sampled_df = combined_df.sample(n=sample_size, random_state=random_state)

    # Configuration
    wave_bands = {
        'Delta': (0.5, 4),
        'Theta': (4, 8),
        'Alpha': (8, 13),
        'Beta': (13, 30),
        'Gamma': (30, 100)
    }
    channels = ['TP9', 'AF7', 'AF8', 'TP10']

    # Sampling rate (adjust based on your actual data)
    sampling_rate = 256  # Hz

    # Results storage
    analysis_results = {
        'time_domain': {},
        'frequency_domain': {},
        'wavelet_analysis': {}
    }

    # Comprehensive Signal Analysis
    for channel in channels:
        channel_results = {}

        # Iterate through wave bands
        for band_name, (low_freq, high_freq) in wave_bands.items():
            # Select specific band and channel
            signal_column = f'{band_name}_{channel}'
            signal_data = sampled_df[signal_column].values

            # 1. Time Domain Analysis
            time_domain_analysis = {
                'mean': np.mean(signal_data),
                'std': np.std(signal_data),
                'median': np.median(signal_data),
                'min': np.min(signal_data),
                'max': np.max(signal_data),
                'skewness': stats.skew(signal_data),
                'kurtosis': stats.kurtosis(signal_data)
            }

            # 2. Frequency Domain Analysis with Fourier Transform
            # Perform FFT
            n = len(signal_data)
            fft_vals = fft(signal_data)
            frequencies = fftfreq(n, 1/sampling_rate)

            # Compute power spectrum
            power_spectrum = np.abs(fft_vals)**2

            # Normalize and filter to relevant frequencies
            positive_freq_mask = frequencies > 0
            normalized_power = power_spectrum[positive_freq_mask] / np.max(power_spectrum[positive_freq_mask])
            filtered_freqs = frequencies[positive_freq_mask]

            # 3. Advanced Frequency Analysis
            freq_domain_analysis = {
                'total_power': np.sum(power_spectrum),
                'dominant_frequency': filtered_freqs[np.argmax(normalized_power)],
                'frequency_distribution': {
                    'low_freq_power': np.sum(normalized_power[filtered_freqs < 10]),
                    'mid_freq_power': np.sum(normalized_power[(filtered_freqs >= 10) & (filtered_freqs < 30)]),
                    'high_freq_power': np.sum(normalized_power[filtered_freqs >= 30])
                }
            }

            # Store results
            channel_results[band_name] = {
                'time_domain': time_domain_analysis,
                'frequency_domain': freq_domain_analysis,
                'fft_data': {
                    'frequencies': filtered_freqs,
                    'power_spectrum': normalized_power
                }
            }

        # Store channel results
        analysis_results['time_domain'][channel] = channel_results
        analysis_results['frequency_domain'][channel] = channel_results

    return analysis_results, sampled_df

def visualize_signal_analysis(analysis_results, sampled_df):
    """
    Create comprehensive visualizations of EEG signal analysis

    Parameters:
    -----------
    analysis_results : dict
        Analysis results from advanced_signal_analysis
    sampled_df : pandas DataFrame
        Sampled dataframe for additional context
    """
    # Visualization configuration
    channels = list(analysis_results['time_domain'].keys())
    wave_bands = list(list(analysis_results['time_domain'].values())[0].keys())

    # Create a multi-panel figure with comprehensive analysis
    fig, axs = plt.subplots(len(channels), 3, figsize=(25, 20))
    plt.subplots_adjust(hspace=0.4, wspace=0.3)

    # Color palette for different wave bands
    color_palette = {
        'Delta': 'blue',
        'Theta': 'green',
        'Alpha': 'red',
        'Beta': 'purple',
        'Gamma': 'orange'
    }

    # Iterate through channels
    for i, channel in enumerate(channels):
        # 1. Time Domain Plot
        axs[i, 0].set_title(f'{channel} - Time Domain Analysis')
        for band in wave_bands:
            signal_data = sampled_df[f'{band}_{channel}'].values
            axs[i, 0].plot(signal_data, label=band, color=color_palette[band], alpha=0.7)
        axs[i, 0].set_xlabel('Sample Index')
        axs[i, 0].set_ylabel('Amplitude')
        axs[i, 0].legend()

        # 2. Frequency Domain Plot
        axs[i, 1].set_title(f'{channel} - Frequency Domain Analysis')
        for band in wave_bands:
            freq_data = analysis_results['frequency_domain'][channel][band]['fft_data']
            axs[i, 1].plot(freq_data['frequencies'], freq_data['power_spectrum'],
                           label=band, color=color_palette[band], alpha=0.7)
        axs[i, 1].set_xlabel('Frequency (Hz)')
        axs[i, 1].set_ylabel('Normalized Power')
        axs[i, 1].set_xlim(0, 50)  # Focus on relevant frequency range
        axs[i, 1].legend()

        # Frequency Power Distribution Visualization
        freq_distribution = []
        band_labels = []

        for band in wave_bands:
            # Extract frequency powers
            low_freq_power = analysis_results['frequency_domain'][channel][band]['frequency_domain']['frequency_distribution']['low_freq_power']
            mid_freq_power = analysis_results['frequency_domain'][channel][band]['frequency_domain']['frequency_distribution']['mid_freq_power']
            high_freq_power = analysis_results['frequency_domain'][channel][band]['frequency_domain']['frequency_distribution']['high_freq_power']

            # Calculate median power for the band
            median_band_power = np.median([low_freq_power, mid_freq_power, high_freq_power])

            freq_distribution.append(median_band_power)
            band_labels.append(band)

        # Plot the median frequency power for each band
        axs[i, 2].bar(band_labels, freq_distribution)
        axs[i, 2].set_title(f'{channel} - Median Frequency Power Distribution')
        axs[i, 2].set_xlabel('Brainwave Bands')
        axs[i, 2].set_ylabel('Median Normalized Power')
        axs[i, 2].tick_params(axis='x', rotation=45)


    plt.suptitle('Comprehensive EEG Signal Analysis', fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
# Main execution function
def main(combined_df):
    # Perform advanced signal analysis
    analysis_results, sampled_df = advanced_signal_analysis(combined_df)

    # Visualize the results
    visualize_signal_analysis(analysis_results, sampled_df)

    return analysis_results, sampled_df

In [ ]:
results, sampled_data = main(combined_df)

# Comprehensive Signal Analysis:


**Time Domain Analysis**

Statistical measures (mean, median, standard deviation)
Distribution characteristics (skewness, kurtosis)


**Frequency Domain Analysis**

Fast Fourier Transform (FFT)
Power spectrum calculation
Dominant frequency identification
Frequency band power distribution



**Visualization Techniques**

Three-panel plot for each channel:

a) Time Domain Signal

Shows raw signal for different wave bands
Color-coded by wave band

b) Frequency Domain Analysis

Normalized power spectrum
Reveals frequency content of signals

c) Frequency Power Distribution

Bar chart showing power across different frequency ranges
Breaks down power for low, mid, and high frequencies



Fourier Transform Explanation:
The Fast Fourier Transform (FFT) converts time-domain signals into frequency-domain representations.

# Peak and Change Direction Detection

In [ ]:
def advanced_signal_analysis(combined_df, sample_percentage=0.25, random_state=392):
    """
    Placeholder for the original signal analysis function.
    """
    sampled_df = combined_df.sample(frac=sample_percentage, random_state=random_state)
    analysis_results = {
        'time_domain': {
            'channel_1': {
                'delta': sampled_df['delta_channel_1'].values,
                'alpha': sampled_df['alpha_channel_1'].values,
                'beta': sampled_df['beta_channel_1'].values,
                'gamma': sampled_df['gamma_channel_1'].values
            },
            'channel_2': {
                'delta': sampled_df['delta_channel_2'].values,
                'alpha': sampled_df['alpha_channel_2'].values,
                'beta': sampled_df['beta_channel_2'].values,
                'gamma': sampled_df['gamma_channel_2'].values
            }
        }
    }
    return analysis_results, sampled_df


def detect_peaks_and_change_direction(signal_data, prominence=0.5, width=5):
    """
    Detect peaks, valleys, and direction changes in the signal.
    """
    # Detect peaks and valleys
    peaks, peak_properties = signal.find_peaks(signal_data, prominence=prominence, width=width)
    valleys, valley_properties = signal.find_peaks(-signal_data, prominence=prominence, width=width)

    # Calculate signal derivatives to identify direction changes
    derivatives = np.gradient(signal_data)
    direction_changes = []
    prev_derivative_sign = np.sign(derivatives[0])
    change_start = 0

    for i in range(1, len(derivatives)):
        current_derivative_sign = np.sign(derivatives[i])
        if current_derivative_sign != prev_derivative_sign:
            direction_changes.append({
                'start': change_start,
                'end': i,
                'previous_direction': 'increasing' if prev_derivative_sign > 0 else 'decreasing',
                'new_direction': 'increasing' if current_derivative_sign > 0 else 'decreasing'
            })
            change_start = i
            prev_derivative_sign = current_derivative_sign

    # Analyze peak characteristics
    peak_analysis = {
        'peak_indices': peaks,
        'peak_heights': signal_data[peaks],
        'peak_prominences': peak_properties['prominences'],
        'valley_indices': valleys,
        'valley_heights': signal_data[valleys]
    }

    metrics = {
        'total_peaks': len(peaks),
        'total_valleys': len(valleys),
        'total_direction_changes': len(direction_changes),
        'mean_peak_height': np.mean(signal_data[peaks]) if len(peaks) > 0 else None,
        'mean_valley_height': np.mean(signal_data[valleys]) if len(valleys) > 0 else None
    }

    return {
        'peak_analysis': peak_analysis,
        'direction_changes': direction_changes,
        'metrics': metrics
    }


def integrate_change_detection(advanced_signal_analysis_func):
    """
    Decorator to extend signal analysis with peak and change detection.
    """
    def wrapper(combined_df, sample_percentage=0.1, random_state=42):
        analysis_results, sampled_df = advanced_signal_analysis_func(combined_df, sample_percentage, random_state)

        change_detection_results = {}
        for channel in analysis_results['time_domain'].keys():
            channel_results = {}
            for band in analysis_results['time_domain'][channel].keys():
                signal_column = f'{band}_{channel}'
                signal_data = sampled_df[signal_column].values
                change_detection = detect_peaks_and_change_direction(signal_data)

                # Plot visualization for this signal
                visualize_signal_analysis(signal_data, change_detection, title=f'Signal Analysis for {signal_column}')

                channel_results[band] = change_detection

            change_detection_results[channel] = channel_results

        analysis_results['change_detection'] = change_detection_results
        return analysis_results, sampled_df

    return wrapper


def visualize_signal_analysis(signal_data, change_detection, title="Signal Analysis"):
    """
    Generate visualizations for peaks, valleys, and direction changes.
    """
    fig, axes = plt.subplots(2, 1, figsize=(10, 8))

    # Extract peak and valley information
    peaks = change_detection['peak_analysis']['peak_indices']
    valleys = change_detection['peak_analysis']['valley_indices']
    direction_changes = change_detection['direction_changes']
    derivatives = np.gradient(signal_data)

    # 1️⃣ Plot signal with peaks and valleys
    axes[0].plot(signal_data, label='Signal', color='blue', alpha=0.6)
    axes[0].scatter(peaks, signal_data[peaks], color='red', label='Peaks', zorder=5)
    axes[0].scatter(valleys, signal_data[valleys], color='green', label='Valleys', zorder=5)
    axes[0].set_title(f'{title} - Peaks and Valleys')
    axes[0].legend()
    axes[0].set_xlabel('Time')
    axes[0].set_ylabel('Amplitude')

    # 2️⃣ Plot the derivative with direction change points
    axes[1].plot(derivatives, label='Derivative', color='purple', alpha=0.6)
    for change in direction_changes:
        axes[1].axvline(x=change['start'], color='red', linestyle='--', alpha=0.7, label='Direction Change')
    axes[1].set_title(f'{title} - Direction Changes')
    axes[1].legend()
    axes[1].set_xlabel('Time')
    axes[1].set_ylabel('Derivative')

    plt.tight_layout()
    plt.show()


# Apply the decorator after the function is defined
advanced_signal_analysis = integrate_change_detection(advanced_signal_analysis)


# Generate sample data for delta, alpha, beta, gamma for 2 channels
combined_df = pd.DataFrame({
    'delta_channel_1': np.sin(np.linspace(0, 10, 100)) + np.random.normal(0, 0.1, 100),
    'alpha_channel_1': np.cos(np.linspace(0, 10, 100)) + np.random.normal(0, 0.1, 100),
    'beta_channel_1': np.sin(2 * np.linspace(0, 10, 100)) + np.random.normal(0, 0.1, 100),
    'gamma_channel_1': np.sin(4 * np.linspace(0, 10, 100)) + np.random.normal(0, 0.1, 100),
    'delta_channel_2': np.cos(np.linspace(0, 10, 100)) + np.random.normal(0, 0.1, 100),
    'alpha_channel_2': np.sin(np.linspace(0, 10, 100)) + np.random.normal(0, 0.1, 100),
    'beta_channel_2': np.sin(3 * np.linspace(0, 10, 100)) + np.random.normal(0, 0.1, 100),
    'gamma_channel_2': np.sin(5 * np.linspace(0, 10, 100)) + np.random.normal(0, 0.1, 100)
})

In [ ]:
analysis_results, sampled_df = advanced_signal_analysis(combined_df, sample_percentage=0.5)

# Signal Analysis Code:

The primary goals of this code are to:
- Sample and analyze EEG signals
- Extract meaningful characteristics from different brain wave bands
- Perform time and frequency domain analysis
- Visualize complex signal characteristics
- Detect peaks and changes in signal direction

## 2. Detailed Function Breakdown

### `advanced_signal_analysis()` Function

This is the core analysis function with several key steps:

1. **Sampling Strategy**
   - Randomly samples a percentage of the original data
   - Ensures reproducibility with a fixed random seed
   - Helps manage large datasets by working with a representative subset

2. **Wave Band Configuration**
   - Defines brain wave bands with their frequency ranges:
     - Delta (0.5-4 Hz): Associated with deep sleep
     - Theta (4-8 Hz): Linked to meditation, memory
     - Alpha (8-13 Hz): Relaxation and calm awareness
     - Beta (13-30 Hz): Active thinking, concentration
     - Gamma (30-100 Hz): High-level cognitive processing

3. **Analysis Dimensions**
   - **Time Domain Analysis**:
     - Calculates statistical measures like mean, median, standard deviation
     - Computes signal skewness and kurtosis
     - Provides insights into signal distribution and variability

   - **Frequency Domain Analysis**:
     - Uses Fast Fourier Transform (FFT) to convert time-based signal to frequency components
     - Identifies dominant frequencies
     - Breaks down power distribution across frequency ranges

### `visualize_signal_analysis()` Function

Creates a comprehensive visualization with three panels for each channel:
1. Time Domain Plot: Shows signal amplitude over time
2. Frequency Domain Plot: Displays normalized power across frequencies
3. Frequency Power Distribution: Bar chart of power across different frequency bands

### New `detect_peaks_and_change_direction()` Function

This function (in the new artifact) adds advanced signal change detection:

1. **Peak Detection**:
   - Finds local maxima (peaks) and minima (valleys)
   - Uses prominence and width parameters to filter significant peaks
   
2. **Direction Change Detection**:
   - Calculates signal derivatives
   - Identifies regions where signal direction shifts
   - Tracks transitions between increasing and decreasing trends

3. **Metrics Computation**:
   - Counts total peaks and valleys
   - Calculates average peak and valley heights
   - Provides comprehensive change analysis

## 3. Signal Processing Techniques Used

### Fourier Transform
- Converts time-domain signal to frequency-domain representation
- Helps understand signal composition
- Reveals hidden frequency components

### Derivative Analysis
- Measures rate of change in the signal
- Identifies inflection points and trend shifts
- Provides insight into signal dynamics



